## CNN for Image Recognition with MNIST Dataset


### Sowmya Askani

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
#from keras.datasets import mnist
from tensorflow.examples.tutorials.mnist import input_data

### Loading MNIST dataset

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
mnist.train.images

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
mnist.train.images.shape

(55000, 784)

In [7]:
mnist.test.images

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [8]:
mnist.test.images.shape

(10000, 784)

In [9]:
mnist.train.labels

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [10]:
mnist.train.labels.shape

(55000, 10)

In [11]:
mnist.test.labels

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
mnist.test.labels.shape

(10000, 10)

### Initialize Weights in Filter 

In [13]:
# This function returns a tf.Variable used to store weights in a filter
# This variable is initialized with values that can be used to initialize weights 
# The values are random numbers

def initialize_weights(filter_shape):
    
    init_random_dist = tf.truncated_normal(filter_shape,stddev=0.1)
    
    return (tf.Variable(init_random_dist))

### Initialize Bias

In [14]:
# This function returns a tf.Variable used to store bias
# This variable is initialized with values that can be usedto initialized bias
# The value is initialized to 0.1

def initialize_bias(bias_shape):
    
    initial_bias_vals = tf.constant(0.1,shape=bias_shape)
    
    return (tf.Variable(initial_bias_vals))

### Set Up Convolution Layer and Perform Convolution Computation: Dot Product (x * W)

In [15]:
def create_convolution_layer_and_compute_dot_product (inputs, filter_shape):
    #Initialize the weights in the filter
    filter_initialized_with_weights = initialize_weights(filter_shape)
    
    #Create a Convolution Layer
    conv_layer_outputs = tf.nn.conv2d(inputs,filter_initialized_with_weights, strides=[1,1,1,1],padding ='SAME')
    
    #Return the convolution layer outputs
    return (conv_layer_outputs)

### Set Up Relu Layer and perform Computation: Dot Product + Bias (x.W + b)

In [16]:
def create_relu_layer_and_compute_dotproduct_plus_b(inputs, filter_shape):
    
    #Initialize bias for each input channel
    b = initialize_bias([filter_shape[3]])
    
    # Perform the computation first by adding: inputs (x * W) + b
    # Create a RelU layer associated with the preceding convolution layer
    relu_layer_outputs = tf.nn.relu(inputs+b)
    
    # Return the outputs of the ReLU layer
    return (relu_layer_outputs)

### Set Up a Pooling Layer and Reduce Spatial Size

In [17]:
def create_max_pool(inputs):
    
    #Create a pooling layer
    pooling_layer_outputs = tf.nn.max_pool(inputs, ksize=[1,2,2,1],strides = [1,2,2,1],padding='SAME')
    
    # Return the pooling Layer
    return (pooling_layer_outputs)

### Set Up Fully Connected Layer and Perform Computation: (inputs * Weights) + Bias

In [18]:
def create_fully_connected_layer_and_compute_dotproduct_plus_bias (inputs, output_size):
    
    #Get the number of input channels from the input
    #Inputs: the outputs fro  the preceding Layer or previous operation like reshaping 
    input_size = int(inputs.get_shape()[1])
    
    #Initialize the weights of the filter of the FC layet
    # Filter shape [in_channels, out_channels]
    # Each weight for one filter cell
    W = initialize_weights([input_size, output_size])
    
    #Initialize the bias: each bias one output channel
    b = initialize_bias([output_size])
    
    #Perform the computation for FC layer and then add bias to get the result
    fcbias_outputs=tf.matmul(inputs,W)+b
    
    # Return the results: outputs
    return (fcbias_outputs)

### Phase 1: Build the convolutional Neural Network

### Create placeholders for inputs and labels: x andy_true

In [19]:
# Placeholder
# Create a placeholder for the inputs data: x
x = tf.placeholder(tf.float32,shape=[None, 784])

In [20]:
# Create a placeholder for the labes of the inputs data: y_true
y_true = tf.placeholder(tf.float32,shape=[None, 10])

### Reshape the input placeholder x

In [21]:
# Prepare feeding inputs into the 1st conv layer
# Reshape the input x: A placeholder
# From 1D array (vector) --> original input shape: 4D-input: [batch, H, W, depth channels]
# Depth = color channels: gray scale = 1
# Reshaped inputs: x_image: [1, 28, 28, 1]

x_image = tf.reshape(x,[-1,28,28,1])

### Create 1st convolutional layer, ReLU layer, and perform computation: x*W + b

In [24]:
#Create a first convolution layer
conv_layer1=create_convolution_layer_and_compute_dot_product(x_image,filter_shape=[5,5,1,32])

In [25]:
#Create the ReLU layer for the 1st convolution layer
# Accept the outputs from the 1st conv layer as the inputs
# Perform the computation at the layer and rfeturn the output to the layer
conv_relu_layer1=create_relu_layer_and_compute_dotproduct_plus_b(conv_layer1,filter_shape=[5,5,1,32])

### Create 1st pooling layer and reduce spatial size

In [26]:
pooling_layer1 = create_max_pool(conv_relu_layer1)

### Create 2nd convolution Layer, RelU Layer and perform computation: x*W + b

In [28]:
# Create a 2nd convolutional layer
conv_layer2=create_convolution_layer_and_compute_dot_product(pooling_layer1,filter_shape=[5,5,32,64])

In [29]:
# Create the ReLU layer for the 2nd convolution layer
conv_relu_layer2=create_relu_layer_and_compute_dotproduct_plus_b(conv_layer2,filter_shape=[5,5,32,64])

### Create 2nd Pooling layer and Reduce Spatial size

In [30]:
pooling_layer2 = create_max_pool(conv_relu_layer2)

### Reshape/Flatten Data Making It Ready to be Fed into 1st FC layer

In [31]:
# Reshape and flatten the output of the 2nd pooling layer
# Prepare to feed the output data into the 1st fully connected layer

poolig_layer2_flatten = tf.reshape(pooling_layer2, [-1,7*7*64])

### create 1st FC Layer, ReLU layer, and Output Data to Dropout Layer

In [33]:
# First, Create the FC (fully connected) layer
fc_layer_1_outputs = create_fully_connected_layer_and_compute_dotproduct_plus_bias(poolig_layer2_flatten, output_size=1024)

In [34]:
# Create the ReLU layer of the 1st FC layer
fc_relu_layer_1_outputs = tf.nn.relu(fc_layer_1_outputs)

### Create Dropout Layer and Dropout a Fractiom of Output Randomly

In [35]:
dropout_hold_prob = tf.placeholder(tf.float32)
dropout_outputs = tf.nn.dropout(fc_relu_layer_1_outputs, keep_prob=dropout_hold_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Create final FC layer, compute (x.W+b), and Produce final outputs

In [37]:
y_pred = create_fully_connected_layer_and_compute_dotproduct_plus_bias(dropout_outputs,output_size=10)

### Define Loss Function and Calculate softmax cross entropy loss

In [38]:
softmax_cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true,logits=y_pred)
cross_entropy_mean = tf.reduce_mean(softmax_cross_entropy_loss)

### Create an Optimizer to optimize CNN Model and set learning rate

In [39]:
# Get an ADAM optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

### Create a trainer to traning CNN Model

In [40]:
#Create a CNN model trainer that can train the model
# And optmize the model by minimizing the softmax cross_entropy loss

cnn_trainer=optimizer.minimize(cross_entropy_mean)

## Train and Test CNN Deep learning model on MNIST dataset

### Create a variable initializer to Initialize all variables

In [41]:
vars_initializer = tf.global_variables_initializer()
steps=5000

### Run tf.Sessions() to Train and Test Deep Learning CNN Model

In [42]:
with tf.Session() as sess:
    sess.run(vars_initializer)
    results=[]
    for i in range(steps):
        batch_x,batch_y = mnist.train.next_batch(50)
        sess.run(cnn_trainer,feed_dict = {x:batch_x, y_true:batch_y,dropout_hold_prob:0.5})
        if i%100==0:
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            test_accuracy = sess.run(acc, feed_dict = {x:mnist.test.images,y_true:mnist.test.labels,dropout_hold_prob:1.0})
            results.append(test_accuracy)
            print(f"ON STEP:{i} Accuracy:{round((test_accuracy*100),2)}")
            print("\n")


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   OMP_NUM_THREADS=8

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_HAND_THREAD=false
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_IN

ON STEP:0 Accuracy:15.64


ON STEP:100 Accuracy:95.1


ON STEP:200 Accuracy:96.36


ON STEP:300 Accuracy:97.32


ON STEP:400 Accuracy:97.13


ON STEP:500 Accuracy:97.24


ON STEP:600 Accuracy:98.0


ON STEP:700 Accuracy:97.91


ON STEP:800 Accuracy:98.04


ON STEP:900 Accuracy:98.03


ON STEP:1000 Accuracy:98.34


ON STEP:1100 Accuracy:98.53


ON STEP:1200 Accuracy:98.56


ON STEP:1300 Accuracy:98.52


ON STEP:1400 Accuracy:98.72


ON STEP:1500 Accuracy:98.37


ON STEP:1600 Accuracy:98.74


ON STEP:1700 Accuracy:98.63


ON STEP:1800 Accuracy:98.61


ON STEP:1900 Accuracy:98.65


ON STEP:2000 Accuracy:98.91


ON STEP:2100 Accuracy:98.61


ON STEP:2200 Accuracy:98.9


ON STEP:2300 Accuracy:98.72


ON STEP:2400 Accuracy:98.73


ON STEP:2500 Accuracy:98.78


ON STEP:2600 Accuracy:98.83


ON STEP:2700 Accuracy:98.69


ON STEP:2800 Accuracy:98.65


ON STEP:2900 Accuracy:98.83


ON STEP:3000 Accuracy:98.97


ON STEP:3100 Accuracy:98.74


ON STEP:3200 Accuracy:99.05


ON STEP:3300 Accuracy:98.

In [43]:
round(np.mean(results)*100,2)

96.84